<a href="https://colab.research.google.com/github/akshatsingh523/AIML-projects/blob/main/t5_base_QLoRa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install evaluate
!pip install rouge
!pip install langchain
!pip install accelerate
!pip install bitsandbytes
!pip install sentencepiece
!pip install -U nlp
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install rouge_score
!pip install evaluate
!pip install dill==0.3.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import json
import torch.nn as nn
import nlp
import pandas as pd
import numpy as np
import transformers
import evaluate
import dataclasses

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import TrainingArguments, Trainer
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass, field
from typing import Dict, List, Optional

import warnings
warnings.filterwarnings("ignore")

In [ ]:

from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base",
                                               quantization_config=bnb_config,
                                               device_map="auto")

Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at google/flan-t5-base and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Save the model's parameters to a file
model_id = "model_large"                             # give your model name to avoid confusion

model_path = model_id
torch.save(model.state_dict(), model_path)

# Check the size of the saved model file
import os
model_file_size_bytes = os.path.getsize(model_path)
model_file_size_mb = model_file_size_bytes / (1024 * 1024)  # Convert to MB
model_file_size_gb = model_file_size_mb / 1024  # Convert to GB

print(f"Model Size: {model_file_size_gb:.4f} GB")

Model Size: 0.3075 GB


### Data Pre processing

In [ ]:
# process the examples in input and target text format and the eos token at the end
def add_eos_to_examples(example):
    example['input_text'] = 'question: %s  context: %s </s>' % (example['question'], example['context'])
    example['target_text'] = '%s </s>' % example['answers']['text'][0]
    return example

# tokenize the examples
def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['input_text'], pad_to_max_length=True, max_length=512)
    target_encodings = tokenizer.batch_encode_plus(example_batch['target_text'], pad_to_max_length=True, max_length=512)

    encodings = {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'decoder_input_ids': target_encodings['input_ids'],
        'decoder_attention_mask': target_encodings['attention_mask']
    }

    return encodings

In [ ]:
# load train and validation split of squad
train_dataset  = nlp.load_dataset('squad', split=nlp.Split.TRAIN)
valid_dataset = nlp.load_dataset('squad', split=nlp.Split.VALIDATION)


# map add_eos_to_examples function to the dataset example wise
train_dataset = train_dataset.map(add_eos_to_examples)
# map convert_to_features batch wise
train_dataset = train_dataset.map(convert_to_features, batched=True)

valid_dataset = valid_dataset.map(add_eos_to_examples, load_from_cache_file=False)
valid_dataset = valid_dataset.map(convert_to_features, batched=True, load_from_cache_file=False)


# set the tensor type and the columns which the dataset should return
columns = ['input_ids', 'decoder_input_ids', 'attention_mask', 'decoder_attention_mask']
train_dataset.set_format(type='torch', columns=columns)
valid_dataset.set_format(type='torch', columns=columns)

  0%|          | 0/87599 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


  0%|          | 0/88 [00:00<?, ?it/s]

  0%|          | 0/10570 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

In [ ]:
len(train_dataset), len(valid_dataset)

(87599, 10570)

In [ ]:
@dataclass
class T2TDataCollator():
  def __call__(self, batch: List) -> Dict[str, torch.Tensor]:
    """
    Take a list of samples from a Dataset and collate them into a batch.
    Returns:
    A dictionary of tensors
    """

    input_ids = torch.stack([example['input_ids'] for example in batch])
    lm_labels = torch.stack([example['decoder_input_ids'] for example in batch])
    lm_labels[lm_labels[:, :] == 0] = -100
    attention_mask = torch.stack([example['attention_mask'] for example in batch])
    decoder_attention_mask = torch.stack([example['decoder_attention_mask'] for example in batch])

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': lm_labels,
        'decoder_attention_mask': decoder_attention_mask
    }

### QLoRa

In [ ]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType

model.gradient_checkpointing_enable()
# prepare int-8 model for training
model = prepare_model_for_kbit_training(model)

# Define LoRA Config
lora_config = LoraConfig(r=8,
                        lora_alpha=64,
                        target_modules=["q", "v"],
                        # lora_dropout=0.05,
                        bias="none",
                        task_type="SEQ_2_SEQ_LM"
)

# add LoRA adaptor
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 884,736 || all params: 248,462,592 || trainable%: 0.3560841867092814


### Model Training

In [ ]:

import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments


# Define training args
training_args = Seq2SeqTrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=200,
        learning_rate=2e-4,
        fp16=False,
        logging_steps=10,
        output_dir="outputs",

)
# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),

)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!


In [ ]:
trainer.train().metrics

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,12.177500
20,10.887500
30,9.502200
40,8.471100
50,7.778200
60,7.468300
70,7.300500
80,7.092400
90,7.086600
100,7.003300


{'train_runtime': 339.7956,
 'train_samples_per_second': 2.354,
 'train_steps_per_second': 0.589,
 'total_flos': 352841210265600.0,
 'train_loss': 7.595592269897461,
 'epoch': 0.01}

In [ ]:
def get_answer(question, context):
  input_text = "question: %s  context: %s" % (question, context)
  features = tokenizer([input_text], return_tensors='pt')

  output = model.generate(input_ids=features['input_ids'],
               attention_mask=features['attention_mask'])

  return tokenizer.decode(output[0], skip_special_tokens=True)

context = "Manuel have created RuPERTa-base with the support of HF-Transformers and Google"
question = "Who has supported Manuel?"

get_answer(question, context)

'HF-Transputers and Google'

In [ ]:
questions = ["What is the Invoice number?",
             "What is the Invoice date?",
             "What is the total amount?",
             "State the From address of Invoice?",
             "State the To address of Invoice?",
             "What is the order number?"
             ]

context = "[Document(page_content='Invoice\n\nInvoice Number\n\nINV-3337\n\nFrom: DEMO - Sliced Invoices Suite 5A-1204 123 Somewhere Street Your City AZ 12345 admin@slicedinvoices.com\n\nOrder Number\n\n12345\n\nInvoice Date\n\nJanuary 25, 2016\n\nDue Date\n\nJanuary 31, 2016\n\nTotal Due\n\n$93.50\n\nTo: Test Business 123 Somewhere St Melbourne, VIC 3000 test@test.com\n\nP aid\n\nHrs/Qty\n\nService\n\nRate/Price\n\nAdjust\n\nSub Total\n\nWeb Design This is a sample description...\n\n1.00\n\n$85.00\n\n0.00%\n\n$85.00\n\nSub Total\n\n$85.00\n\nTax\n\n$8.50\n\nTotal\n\n$93.50\n\nANZ Bank ACC # 1234 1234 BSB # 4321 432\n\nPayment is due within 30 days from date of invoice. Late payment is subject to fees of 5% per month. Thanks for choosing DEMO - Sliced Invoices | admin@slicedinvoices.com Page 1/1', metadata={'source': 'invoice.pdf'})]"
for qn in questions:
  print(get_answer(qn, context))


Invoice Number 12345 12345 - 12345 - 12345
'
AB# 123'AB# 123'AB# '
[[Dem - Sliced Invoices]]   
'''''''''
12345


In [ ]:
questions = ["What is the capital of France?",
             "How many days are there I a week?",
             "What is the largest planet in our solar system?",
             "What is the tallest mountain in the world?",
             "What is the main language in India?",
             "Who is the author of Harry Potter book series?"
             ]

context = ""
for qn in questions:
  print(get_answer(qn,context))

France
I'm a week
a helium
The highest mountain in the world is Mount Everest, which is the highest peak in the world
English Language in India
Harry Potter


In [ ]:
!pip install pynvml
import psutil
import pynvml


# RAM Usage
ram = psutil.virtual_memory()
ram_usage = ram.used / (1024 * 1024 * 1024)
print("RAM Usage:", ram_usage, "GB")
print()

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:',(round(torch.cuda.memory_allocated(0)/1024**3,1) + (round(torch.cuda.memory_cached(0)/1024**3,1))), 'GB')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')
print()

# Disk Usage
disk_usage = psutil.disk_usage('/')
print("Disk Usage:", (disk_usage.used / (1024 * 1024 * 1024)-24.2), "GB")

RAM Usage: 3.9375839233398438 GB

Using device: cuda
Tesla T4
Memory Usage: 1.2000000000000002 GB
Allocated: 0.4 GB
Cached:    0.8 GB

Disk Usage: 4.000870513916016 GB
